<a href="https://colab.research.google.com/github/VenGr0/RAG-system/blob/main/%D0%94%D0%BE%D0%BC%D0%B0%D1%88%D0%BD%D1%8F%D1%8F_%D1%80%D0%B0%D0%B1%D0%BE%D1%82%D0%B0_%E2%84%9631.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install openai llama_index llama-index-postprocessor-longllmlingua llmlingua

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 375.1/375.1 kB 14.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.4/76.4 kB 7.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 7.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 318.9/318.9 kB 26.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 69.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 63.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 75.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 56.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 187.4/187.4 kB 18.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.8/295.8 kB 25.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 5.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.3/49.3 kB 4.8 MB/s eta 0:00:00
  Attempting uninstall: tenacity
    Found existing inst

In [ ]:
from llama_index.core import VectorStoreIndex, SimpleDirectoryReader
from llama_index.embeddings.openai import OpenAIEmbedding
from llama_index.llms.openai import OpenAI
from llama_index.core.postprocessor import LLMRerank, LongContextReorder
from llama_index.postprocessor.longllmlingua import LongLLMLinguaPostprocessor
from llama_index.core.query_engine import TransformQueryEngine
from llama_index.core.indices.query.query_transform import HyDEQueryTransform
from llama_index.core import Settings, Document
from llama_index.core.node_parser import SentenceSplitter
from llama_index.core.extractors import TitleExtractor
from llama_index.core.ingestion import IngestionPipeline

import getpass
import os
from IPython.display import Markdown, display

# Запрос API ключа OpenAI
os.environ["OPENAI_API_KEY"] = getpass.getpass("Введите OpenAI API Key:")

# Настройка глобальных параметров
Settings.llm = OpenAI(model_name="gpt-3.5-turbo", temperature=0.1, request_timeout=1000, max_retries=3)
Settings.chunk_size = 512


Введите OpenAI API Key:··········


In [ ]:
documents = SimpleDirectoryReader("./data").load_data()

# Создание индекса
index = VectorStoreIndex.from_documents(documents)


In [ ]:
# Движок запросов с ранжированием LLM
query_engine = index.as_query_engine(
    similarity_top_k=10,
    node_postprocessors=[
        LLMRerank(
            choice_batch_size=5,
            top_n=2,
        )
    ],
)

# Постобработка LongLLMLingua
lingua = LongLLMLinguaPostprocessor(
    instruction_str="Given the context, please answer the final question",
    target_token=300,
    rank_method="longllmlingua",
    additional_compress_kwargs={}  # добавлен пустой словарь
)

query_engine_lingua = index.as_query_engine(
    similarity_top_k=10,
    node_postprocessors=[lingua],
)


# Постобработка LongContextReorder
reorder = LongContextReorder()
reorder_engine = index.as_query_engine(
    node_postprocessors=[reorder], similarity_top_k=10
)

# Преобразование запросов с использованием HyDE
hyde = HyDEQueryTransform(include_original=True)
hyde_query_engine = TransformQueryEngine(query_engine, hyde)


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/583 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/746 [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.84M [00:00<?, ?B/s]

...

model.safetensors.index.json:   0%|          | 0.00/26.8k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/9.98G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/3.50G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/200 [00:00<?, ?B/s]

In [ ]:
query_str = "what is the title of the first chapter in the source??"

# Выполнение запроса с ранжированием LLM
response_rerank = query_engine.query(query_str)
display(Markdown(f"<b>LLMRerank Response:</b> {response_rerank}"))

# Выполнение запроса с LongLLMLingua постобработкой
response_lingua = query_engine_lingua.query(query_str)
display(Markdown(f"<b>LongLLMLingua Response:</b> {response_lingua}"))

# Выполнение запроса с LongContextReorder постобработкой
response_reorder = reorder_engine.query(query_str)
display(Markdown(f"<b>LongContextReorder Response:</b> {response_reorder}"))

# Выполнение запроса с HyDE
response_hyde = hyde_query_engine.query(query_str)
display(Markdown(f"<b>HyDE Response:</b> {response_hyde}"))


<b>LLMRerank Response:</b> The title of the first chapter in the source is "Математические идеи, лежащие в основе работы простых нейронных сетей."

We detected that you are passing `past_key_values` as a tuple and this is deprecated and will be removed in v4.43. Please use an appropriate `Cache` class (https://huggingface.co/docs/transformers/v4.41.3/en/internal/generation_utils#transformers.Cache)


<b>LongLLMLingua Response:</b> The title of the first chapter in the source is "open-hide."

<b>LongContextReorder Response:</b> The title of the first chapter in the source is "Mathematical Foundations of Simple Neural Networks."

<b>HyDE Response:</b> The title of the first chapter in the source is "Математические идеи, лежащие в основе работы простых нейронных сетей."

In [ ]:
import logging
import sys

# Настройка логгера
logger = logging.getLogger("llama_index")
logger.setLevel(logging.INFO)  # Установка уровня логирования

# Добавление StreamHandler для вывода в stdout
handler = logging.StreamHandler(sys.stdout)
handler.setLevel(logging.INFO)

# Форматирование логов
formatter = logging.Formatter('%(asctime)s - %(name)s - %(levelname)s - %(message)s')
handler.setFormatter(formatter)

# Добавляем handler к логгеру
logger.addHandler(handler)

def trace_query_engine(engine, query):
    logger.info(f"Query: {query}")
    response = engine.query(query)
    logger.info(f"Response: {response}")
    return response

# Пример трассировки для движка с LongLLMLingua
response_lingua_traced = trace_query_engine(query_engine_lingua, query_str)


2024-09-20 14:59:58,325 - llama_index - INFO - Query: what is the title of the first chapter in the source??


INFO:llama_index:Query: what is the title of the first chapter in the source??


2024-09-20 15:00:02,411 - llama_index - INFO - Response: The title of the first chapter in the source is "open-hide."


INFO:llama_index:Response: The title of the first chapter in the source is "open-hide."


Анализ данных с трассировкой: Трассировка показывает, какие данные отправляются в LLM, помогая выявлять и оптимизировать процесс генерации ответов.

Эффективность постобработок:

LLMRerank улучшает релевантность ответов, отсекая менее подходящие варианты.

LongContextReorder улучшает структуру ответа, переформатируя контекст.

LongLLMLingua сжимает контекст, делая ответы более компактными и точными.

Преимущества RAG-системы: RAG-система эффективно сочетает поиск по тексту с генеративными возможностями LLM, повышая точность и релевантность ответов. Постобработки позволяют адаптировать систему под конкретные задачи.